In [1]:
import os
import glob

In [2]:
pwd = os.getcwd()
download_path = os.path.join(pwd, "downloads")
os.makedirs(download_path, exist_ok=True)

In [3]:
def read_lines_from_txt(file_path="keywords.txt"):
    with open(file_path, "r") as file:
        lines = file.readlines()
        # Remove any trailing newline characters from each line
        lines = (line.strip() for line in lines)
    return lines


print(read_lines_from_txt())

<generator object read_lines_from_txt.<locals>.<genexpr> at 0x779c295d7c30>


In [4]:
def remove_files_in_directory(directory_path):
    # Get a list of all file paths in the directory
    files = glob.glob(os.path.join(directory_path, "*"))

    # Loop through the list and remove each file
    for file_path in files:
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                pass
        except Exception as e:
            pass

In [11]:
async def download(page, button, to_path):
    async with page.expect_download(timeout=60 * 1000) as download_info:
        # Perform the action that initiates download
        # await page.get_by_text("Download file").click()
        await button.click()

    download = await download_info.value
    path_zip = os.path.join(to_path, download.suggested_filename)
    await download.save_as(path_zip)


async def download_the(page, what, to_path):
    await page.wait_for_selector(
        "div.usa-dt-tab__label", state="attached", timeout=60000
    )
    await page.click("button[title='Download']")
    await page.wait_for_selector(
        "div.full-download-modal", state="attached", timeout=60 * 000
    )
    download_window = await page.query_selector("div.full-download-modal")
    button = await download_window.query_selector(f"button:has-text('{what}')")
    await button.click()
    everything = await download_window.query_selector("button[title='Everything']")

    undownloaded_data = None

    try:
        await download(page, everything, to_path=to_path)
    except:
        undownloaded_data = dict()
        undownloaded_data["url"] = await (
            await download_window.query_selector("div.link")
        ).inner_text()
        undownloaded_data["to_path"] = to_path
        print(undownloaded_data)
        await page.reload()
        await page.wait_for_timeout(10 * 1000)
    finally:
        return undownloaded_data


async def download_revisit(page, not_yet_downloaded):
    url = not_yet_downloaded["url"]
    to_path = not_yet_downloaded["to_path"]
    async with page.expect_download(timeout=60 * 60 * 1000) as download_info:
        # Perform the action that initiates download
        # await page.get_by_text("Download file").click()
        await page.goto(url)

    download = await download_info.value
    path_zip = os.path.join(to_path, download.suggested_filename)
    await download.save_as(path_zip)

In [15]:
from playwright.async_api import async_playwright


async def usa_spending_downloader(headless=True):
    print("Starting Downloader")
    p = await async_playwright().start()
    browser = await p.chromium.launch(headless=headless)
    context = await browser.new_context(
        accept_downloads=True,
        user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    )
    page = await context.new_page()
    url = "https://www.usaspending.gov/search/"
    await page.goto(url, timeout=60000)
    await page.wait_for_load_state("networkidle")

    await page.wait_for_selector("div.search-results", state="attached", timeout=60000)

    # Adding keywords
    await page.click("button.filter-toggle__button")

    for k in read_lines_from_txt():
        await page.wait_for_timeout(1000)
        await page.click("input#search")
        await page.fill("input#search", k)
        await page.click("button.keyword-submit")

    filters_container = await page.query_selector("div.fy-columns-container")
    year_checkboxes = await filters_container.query_selector_all(
        "label.fy-option-wrapper"
    )
    submit_button = await page.query_selector("button:has-text('Submit')")
    pwd = os.getcwd()
    download_dir = os.path.join(pwd, "downloads")
    pending_downloads = []
    for i in range(len(year_checkboxes)):
        # setting up year dir
        y = year_checkboxes[i]
        year = (await y.inner_text()).replace(" ", "_")
        download_year_dir = os.path.join(download_dir, year)
        print(download_year_dir)
        os.makedirs(download_year_dir, exist_ok=True)
        remove_files_in_directory(download_year_dir)

        # download
        await y.click()
        await submit_button.click()

        # award
        undownloaded_data = await download_the(
            page, what="Award", to_path=download_year_dir
        )

        await page.wait_for_timeout(3000)
        await page.press("body", "Escape")

        if undownloaded_data:
            pending_downloads.append(undownloaded_data)
            filters_container = await page.query_selector("div.fy-columns-container")
            year_checkboxes = await filters_container.query_selector_all(
                "label.fy-option-wrapper"
            )
            y = year_checkboxes[i]
            submit_button = await page.query_selector("button:has-text('Submit')")

        # transaction
        undownloaded_data = await download_the(
            page, what="Transaction", to_path=download_year_dir
        )

        await page.wait_for_timeout(3000)
        await page.press("body", "Escape")

        if undownloaded_data:
            pending_downloads.append(undownloaded_data)
            filters_container = await page.query_selector("div.fy-columns-container")
            year_checkboxes = await filters_container.query_selector_all(
                "label.fy-option-wrapper"
            )
            y = year_checkboxes[i]
            submit_button = await page.query_selector("button:has-text('Submit')")

        await y.click()

        # break

    n_pending = len(pending_downloads)
    print(f"Pending downloads {n_pending}")
    for to_down in pending_downloads:
        await download_revisit(page, to_down)

    await page.wait_for_timeout(5000)

    await browser.close()
    await p.stop()


await usa_spending_downloader(headless=False)

Starting Downloader
/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2024
/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2023
/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2022
/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2021
{'url': 'https://files.usaspending.gov/generated_downloads/PrimeAwardSummariesAndSubawards_2024-09-08_H20M39S35306157.zip', 'to_path': '/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2021'}
{'url': 'https://files.usaspending.gov/generated_downloads/SubawardsAndPrimeTransactions_2024-09-08_H21M16S09261085.zip', 'to_path': '/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2021'}
/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2020
{'url': 'https://files.usaspending.gov/generated_downloads/PrimeAwardSummariesAndSubawards_2024-09-08_H21M27S36593911.zip', 'to_path': '/home/fsilva/Desktop/projects/usa-spending/downloads/FY_2020'}
{'url': 'https://files.usaspending.gov/generated_downloads/SubawardsAndPrimeTran

/tmp/ipykernel_231694/2506877908.py:90: RuntimeWarning: coroutine 'download_revisit' was never awaited
  download_revisit(page, to_down)
